# <span style="color:orange">Nettoyer des jeux de données pour obtenir une liste de DOI des publications de l'Inserm : Web of Science, Pubmed, HAL</span>

Ce premier notebook sert à nettoyer les différents fichiers obtenus après téléchargement sur le WoS, Pubmed, HAL. Pour savoir quelques requêtes ont été utilisées pour l'Inserm, consulter le fichier "requetes_bdd" dans le dossier. Quelques consignes sont à respecter pour que tout fonctionne :

- Pour le WoS, il suffit de procéder à un téléchargement simple "Fast 5000". Le fichier obtenu, en texte, est illisible et c'est normal, il n'y a rien à changer. Nommer le fichier "wos_lorraine_2016", puis "wos_lorraine_2017"... Ce fichier n'apparaît pas dans le dossier téléchargé depuis Gitlab car les données du Web of Science étant propriétaires, il n'était pas possible de les diffuser librement.
- Pour Pubmed, le téléchargement donne un fichier CSV très peu classé, c'est normal, il n'y a rien à changer. Nommer le fichier "pubmed_inserm_2016", puis "pubmed_inserm_2017"...
- Pour les autres sources de données, on obtient directement une liste de DOI, mais **il faut s'assurer que la colonne s'appelle bien "doi" en minuscules et qu'il n'y a pas de ligne vide**

Il faut télécharger année par année, et toujours nommer les fichiers de la même manière.  <span style="color:red"> **Il est vital de garder l'organisation ici présente (Data > raw > dossier par année) pour que le code fonctionne.**</span>

<span style="color:red">**Si l'on ne dispose pas de certaines données (par exemple, l'établissement n'a pas de données sur les APC ou n'utilise pas le Web of Science), il ne faut pas exécuter les parties de code liées à ces outils. Si l'on ne dispose pas d'extractions du Web of Science, on n'exécute pas toute la partie "Nettoyer les données issues du Web of Science".**</span>

**Il faut modifier les valeurs des variables "etabl" et "annee" dans les cases ci-dessous suivant votre situation.
Dans la variable "etabl" mettre le nom de l'établissement.  
Dans la variable "annee" mettre la dernière année à traiter, par exemple quand les données de 2023 seront ajoutées dans le dossier "raw", il faudra modifier la valeur d'année de la façon suivante: annee = 2023**

Commencer par exécuter les lignes ci-dessous : cliquer sur la ligne puis ensuite sur le bouton "play" de la barre d'outils.

In [1]:
etabl = "inserm"
annee = 2022

In [2]:
import pandas as pd
import csv

## <span style="color:orange">Nettoyer les données issues du Web of Science</span>

### Comprendre comment est structuré le fichier

On ouvre le premier fichier puis on utilise la méthode de liste **split** pour voir une ligne (la 4ème, par exemple).

In [3]:
with open(f"Data/raw/{annee}/wos_{etabl}_{annee}.txt", encoding="utf8") as f:
    wos_string = f.read()

La ligne ci-dessous permet de voir à quoi ressemble la 5ème ligne du fichier (en Python, la numérotation commence à zéro).

In [4]:
wos_string.split("\n")[4]

'J\tPuertas, Rosa Donate; Deflers, Carole; Puech, Frederic; Fischmeister, Rodolphe; Hillaireau, Herve; Pidoux, Guillaume\t\t\t\t\t\t\t\tNovel specific cardiac homing peptides to target failing cardiomyocytes\t\t\t\t\t\t\t\tJOURNAL OF MOLECULAR AND CELLULAR CARDIOLOGY\t\t\t\t173\t\t\t\tS57\tS58\t\t10.1016/j.yjmcc.2022.08.112\t\t\tDEC 31 2022\t2022\t\t\t[Puertas, Rosa Donate; Deflers, Carole; Fischmeister, Rodolphe; Pidoux, Guillaume] Univ Paris Saclay, INSERM UMR S 1180, Signaling & Cardiovasc Pathophysiol Lab, Chatenay Malabry, France; [Puertas, Rosa Donate; Hillaireau, Herve] Univ Paris Saclay, UMR CNRS 8612, Galien Paris Saclay Inst, Chatenay Malabry, France; [Puech, Frederic] Integrated Drug Discovery Res & Dev Sanofi, Chilly Mazarin, France\t\t\t\t24th World Congress of the International-Society-for-Heart-Research\tJUN 12-15, 2022\tInt Soc Heart Res\tBerlin, GERMANY\t0\t\t\t\t\t\t\tCardiac & Cardiovascular Systems; Cell Biology\tWOS:000907054400108\t\t'

### Définir une fonction

#### Définition de la fonction **read_wos** :  
La fonction prend en argument une date de départ et et une date de fin.  
Ces arguments sont optionnels et ont une valeur par défaut, respectivement 2016 et 2022.

Exemple d'utilisation:  
* Un appel à **read_wos()** renverra par défaut, les données de 2016 à 2022 inclue.  
* Un appel à **read_wos(start_year = 2017)** renverra les données de 2017 à 2022 inclue.  
* Un appel à **read_wos(end_year = 2023)** renverra les données de 2016 à 2023 inclue.  
* Un appel à **read_wos(start_year = 2017, end_year = 2018)** renverra les données de 2017 à 2018 inclue.  

On crée une liste vide (wos_per_year) dans laquelle on stockera après les données de toutes les années grâce à la boucle **for**.

La liste vide wos_per_year est remplie à chaque tour de boucle grâce à la méthode **append**.  
Pour avoir la liste de toutes les données sans tri par année, on concatène les fichiers grâce à **pandas.concat**.

In [5]:
def read_wos(start_year = 2016, end_year = 2022):
    wos_per_year = []
    for year in range (start_year, (end_year + 1)):
        wos_df = pd.read_csv(
            f"Data/raw/{year}/wos_{etabl}_{year}.txt", 
            sep="\t",
            quoting=csv.QUOTE_NONE, 
            index_col=False, 
            usecols=['DI'],
            encoding="utf8",
        )
        wos_per_year.append(wos_df)

    full_wos = pd.concat(wos_per_year, ignore_index = True)
    
    return full_wos.rename(columns={'DI':"doi"})

In [6]:
wos_df = read_wos(end_year = annee)

In [7]:
wos_df

,doi
0,10.1016/j.ijpharm.2016.10.016
1,10.1371/journal.pone.0168548
2,10.1103/PhysRevE.94.062413
3,10.1016/j.canlet.2016.09.030
4,10.3389/fimmu.2016.00646
...,...
135585,10.1080/15592294.2020.1864171
135586,10.1007/s12603-020-1547-5
135587,10.1002/hbm.25206
135588,10.1002/hbm.25098


Vérifier que le chiffre total obtenu ici correspond bien au chiffre trouvé dans le Web of Science pour toutes les années. Ne pas hésiter à refaire les extractions d'années précédentes si les chiffres ne correspondent pas : il peut y avoir de nouvelles publications indexées dans le Web of Science.

### Identifier les cellules qui n'ont pas de DOI

Trouver le nombre de lignes qui n'ont pas de DOI :

In [8]:
num_wos_sans_doi = wos_df["doi"].isna().sum()
num_wos_sans_doi

13404

Trouver le nombre de lignes qui ont un DOI correctement formé: utiliser la méthode **match** pour compter le nombre de string commencant par "10." :

In [9]:
num_wos_avec_doi = wos_df["doi"].str.match("10.").sum()
num_wos_avec_doi

122186

Vérifier le nombre de lignes total :

In [10]:
wos_df.size

135590

On veut s'assurer que le total de nombre de lignes sans DOI et avec DOI correctement formés, correspondent au nombre total de lignes lues dans le ficher. 

On utilise la méthode : **assert**. Si aucune erreur n'est renvoyée, c'est que les deux valeurs comparées sont identiques.

In [11]:
expected = wos_df["doi"].size
actual = num_wos_sans_doi + num_wos_avec_doi

assert(expected == actual)

 <span style="color:green"> **Les données du WoS sont à présent nettoyées et stockées dans la variable intitulée "wos_df".** </span>

## <span style="color:orange">Nettoyer les données issues de Pubmed</span>

### Comprendre comment est structuré le fichier

In [12]:
with open(f"Data/raw/{annee}/pubmed_{etabl}_{annee}.csv", encoding='utf-8') as f:
    pubmed_string = f.read()

In [13]:
pubmed_string.split("\n")[1]

'35057580,Effects of Coffee on the Gastro-Intestinal Tract: A Narrative Review and Literature Update,Nehlig A.,Nutrients. 2022 Jan 17;14(2):399. doi: 10.3390/nu14020399.,Nehlig A,Nutrients,2022,2022/01/21,PMC8778943,,10.3390/nu14020399'

In [14]:
pubmed_df = pd.read_csv(f"Data/raw/{annee}/pubmed_{etabl}_{annee}.csv", encoding='utf-8')

In [15]:
pubmed_df.columns

Index(['PMID', 'Title', 'Authors', 'Citation', 'First Author', 'Journal/Book',
       'Publication Year', 'Create Date', 'PMCID', 'NIHMS ID', 'DOI'],
      dtype='object')

**Il y a bien une colonne qui s'appelle "DOI", elle va être interrogée pour trouver directement les DOI.**


In [16]:
def read_pubmed(start_year = 2016, end_year = 2022):
    pubmed_per_year = []
    for year in range (start_year, (end_year + 1)):
        pubmed_df = pd.read_csv(
            f"Data/raw/{year}/pubmed_{etabl}_{year}.csv",
            sep=",", 
            encoding='utf-8',
            usecols=['DOI'],
        )
        pubmed_per_year.append(pubmed_df)
    
    full_pubmed = pd.concat(pubmed_per_year, ignore_index = True)
    return full_pubmed.rename(columns={'DOI':"doi"})

In [17]:
pubmed_df = read_pubmed(end_year = annee)

In [18]:
pubmed_df

,doi
0,10.1016/j.cell.2016.01.043
1,10.1111/liv.13022
2,10.3390/nu8050254
3,10.1186/s13326-016-0078-9
4,10.1101/cshperspect.a022764
...,...
155415,10.1007/s00705-022-05546-z
155416,10.1016/j.phymed.2022.154520
155417,10.1038/s41586-022-05300-0
155418,10.1016/j.jad.2022.12.022


### Vérifier qu'il ne manque pas de DOI

In [19]:
num_pubmed_sans_doi = pubmed_df["doi"].isna().sum()
num_pubmed_sans_doi

727

In [20]:
num_pubmed_avec_doi = pubmed_df["doi"].str.match("10.").sum()
num_pubmed_avec_doi

154693

In [21]:
expected = pubmed_df.size
actual = num_pubmed_sans_doi + num_pubmed_avec_doi

assert(expected == actual)

 <span style="color:green"> **Les données de Pubmed sont à présent nettoyées et stockées dans la variable intitulée "pubmed_df".** </span>

## <span style="color:orange">Ajouter les données de HAL</span>

Grâce aux API, on peut récupérer une grande liste de DOI. Il peut être intéressant de refaire des extractions d'une année sur l'autre quand des campagnes de dépôt massif sont organisées, par exemple.

Ouvrir le fichier et concaténer les différentes années (mettre "2022" dans le code quand les données de 2021 seront injectées) :

In [22]:
def read_hal(start_year = 2016, end_year = 2022):
    hal_per_year = []
    for year in range (start_year, (end_year + 1)):
        hal_df = pd.read_csv(
            f"Data/raw/{year}/hal_{etabl}_{year}.csv", 
            sep="\t", 
            encoding='latin-1')
        hal_per_year.append(hal_df)
    
    return pd.concat(hal_per_year, ignore_index = True)

In [23]:
hal_df = read_hal(end_year = annee)

In [24]:
hal_df

,doi
0,10.1038/mi.2016.68
1,10.19070/2377-1887-SI01002
2,10.1091/mbc.E15-03-0137
3,10.1523/JNEUROSCI.3379-15.2016
4,10.1038/srep33518
...,...
34479,10.1186/s13148-022-01364-x
34480,10.3389/fninf.2022.803934
34481,10.1210/clinem/dgac512
34482,10.1371/journal.ppat.1010799


 <span style="color:green"> **Les données de HAL sont à présent nettoyées et stockées dans la variable intitulée "hal_df".** </span>

## <span style="color:orange">Rassembler les 3 listes de DOI (WoS, PubMed, HAL) et enlever les doublons</span>

### Concaténer les 3 listes de données

In [25]:
all_df = pd.concat([wos_df, pubmed_df, hal_df], ignore_index = True)
all_df

,doi
0,10.1016/j.ijpharm.2016.10.016
1,10.1371/journal.pone.0168548
2,10.1103/PhysRevE.94.062413
3,10.1016/j.canlet.2016.09.030
4,10.3389/fimmu.2016.00646
...,...
325489,10.1186/s13148-022-01364-x
325490,10.3389/fninf.2022.803934
325491,10.1210/clinem/dgac512
325492,10.1371/journal.ppat.1010799


### Enlever les lignes sans DOI

Le résultat correspond à la concaténation de toutes les lignes des 3 fichiers, y compris celles qui ne comprennent pas de DOI.  
On supprime ces lignes sans DOI avec la fonction: **dropna**.  

In [26]:
all_df.dropna(subset = ["doi"], inplace = True, ignore_index = True)
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311363 entries, 0 to 311362
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   doi     311363 non-null  object
dtypes: object(1)
memory usage: 2.4+ MB


Nous avons ici la liste de tous les DOI recensés dans nos 3 sources d'information. Mais attention, il peut y avoir des doublons : les DOI peuvent notamment être écrits en minuscules comme en majuscules.

### Enlever les doublons

Avant d'enlever les doublons, il faut normaliser les DOI et tous les passer en minuscules.

In [27]:
all_df["doi"] = all_df["doi"].str.lower()

Après avoir passé l'ensemble de la DataFrame en minuscules, on peut enlever les doublons.

In [28]:
doi_inserm_final = all_df.drop_duplicates(ignore_index = True)

In [29]:
print(f'Taille avec doublons: {all_df.size}')
print(f'Taille sans doublons: {doi_inserm_final.size}')
print(f'Pourcentage de doublons: {(all_df.size - doi_inserm_final.size) / all_df.size:.2%}')

Taille avec doublons: 311363
Taille sans doublons: 136469
Pourcentage de doublons: 56.17%


In [30]:
doi_inserm_final

,doi
0,10.1016/j.ijpharm.2016.10.016
1,10.1371/journal.pone.0168548
2,10.1103/physreve.94.062413
3,10.1016/j.canlet.2016.09.030
4,10.3389/fimmu.2016.00646
...,...
136464,10.1126/science.abm5847
136465,10.1002/ajpa.24629
136466,10.1093/ve/veac097
136467,10.1126/science.abm0151


In [31]:
print(f'On obtient {doi_inserm_final.size} DOI pour la période 2016-{annee}')

On obtient 136469 DOI pour la période 2016-2022


### Créer dans le dossier "outputs" un fichier CSV avec la liste des DOI

In [32]:
doi_inserm_final.to_csv(f"Data\outputs\last_doi_{etabl}_2016_{annee}.csv", index = False)

Pour obtenir cette liste dans un format Excel classique, saisir le code suivant.

In [33]:
doi_inserm_final.to_excel(f"Data\outputs\last_doi_{etabl}_2016_{annee}.xlsx", index = False)